In [2]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
con = sqlite3.connect('imdb_2022-02-13.db')
cur = con.cursor()

**Проблема:** у одного фильма бывает много жанров.

**Решение:** Делаем в датабазе таблицу с one-hot кодированием жанров

1. Алгоритм кодирования

In [4]:
all_genres_query = """
    SELECT * FROM film_genres
    WHERE title_id = ?
    """

In [7]:
def film_genres_one_hot(title_id):
    title_id_list = [title_id]
    cur.execute(all_genres_query, title_id_list)
    data = cur.fetchall()
    film_genres = []
    for element in data:
        film_genres.append(element[1])
    film_genres_one_hot = []
    film_genres_one_hot.append(title_id)
    for j in range(28):
        if j+1 in film_genres:
            film_genres_one_hot.append(1)
        else:
            film_genres_one_hot.append(0)
    return film_genres_one_hot

Пример работы:

In [8]:
film_genres_one_hot(335)

[335,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0]

In [9]:
len(film_genres_one_hot(335))

29

2. Делаем таблицу

In [10]:
cur.execute("""
CREATE TABLE all_genres (
    title_id INT,
    is_one INT,
    is_two INT,
    is_three INT,
    is_four INT,
    is_five INT,
    is_six INT,
    is_seven INT,
    is_eight INT,
    is_nine INT,
    is_ten INT,
    is_eleven INT,
    is_twelve INT,
    is_thirteen INT,
    is_fourteen INT,
    is_fiveteen INT,
    is_sixteen INT,
    is_seventeen INT,
    is_eighteen INT,
    is_nineteen INT,
    is_twenty INT,
    is_twentyone INT,
    is_twentytwo INT,
    is_twentythree INT,
    is_twentyfour INT,
    is_twentyfive INT,
    is_twentysix INT,
    is_twentyseven INT,
    is_twentyeight INT,
    PRIMARY KEY (title_id)
)
""")

In [11]:
number_of_films_query = '''SELECT MAX(title_id) FROM titles'''
cur.execute(number_of_films_query)
number_of_films = cur.fetchall()[0][0]
print(number_of_films)

18226684


In [12]:
for i in range(1, number_of_films+1):
    data = film_genres_one_hot(i)
    cur.execute("INSERT INTO all_genres VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data)
con.commit()

In [14]:
###  код на маленьком кусочке для скорости
#for i in range(1, 11):
 #   data = film_genres_one_hot(i)
  #  cur.execute("INSERT INTO all_genres VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data)
#con.commit()

**Делаем общую таблицу со всеми данными.**

In [15]:
cur.execute("""
CREATE TABLE all_data (
    title_id INT,
    title TEXT,
    is_adult INT,
    runtime_min INT,
    premiered INT,
    film_type INT,
    is_one INT,
    is_two INT,
    is_three INT,
    is_four INT,
    is_five INT,
    is_six INT,
    is_seven INT,
    is_eight INT,
    is_nine INT,
    is_ten INT,
    is_eleven INT,
    is_twelve INT,
    is_thirteen INT,
    is_fourteen INT,
    is_fiveteen INT,
    is_sixteen INT,
    is_seventeen INT,
    is_eighteen INT,
    is_nineteen INT,
    is_twenty INT,
    is_twentyone INT,
    is_twentytwo INT,
    is_twentythree INT,
    is_twentyfour INT,
    is_twentyfive INT,
    is_twentysix INT,
    is_twentyseven INT,
    is_twentyeight INT,
    votes INT,
    rating REAL,
    PRIMARY KEY (title_id)
)
""")

In [16]:
all_data_query = """
SELECT titles.title_id, title, is_adult, runtime_min, premiered, film_type, is_one, is_two, is_three, is_four, is_five, is_six, is_seven, is_eight, is_nine, is_ten, is_eleven, is_twelve, is_thirteen, is_fourteen, is_fiveteen, is_sixteen, is_seventeen, is_eighteen, is_nineteen, is_twenty, is_twentyone, is_twentytwo, is_twentythree, is_twentyfour, is_twentyfive, is_twentysix, is_twentyseven, is_twentyeight,
 votes, rating
FROM titles
    JOIN film_types ON film_types.id = titles.type
    JOIN all_genres ON all_genres.title_id = titles.title_id
    JOIN rating ON titles.title_id = rating.title_id
"""

In [17]:
cur.execute(all_data_query)
data = cur.fetchall()

In [18]:
cur.executemany("INSERT INTO all_data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", data)
con.commit()

У нас получилась таблица all_data со всеми данными, но как часть .db.

Мы хотим предсказать рейтинг фильма по многим его параметрам.

Делаем линейную регрессию.

In [19]:
import pandas as pd

In [20]:
df = pd.read_sql_query("SELECT * FROM all_data", con)
df

,title_id,title,is_adult,runtime_min,premiered,film_type,is_one,is_two,is_three,is_four,...,is_twentyone,is_twentytwo,is_twentythree,is_twentyfour,is_twentyfive,is_twentysix,is_twentyseven,is_twentyeight,votes,rating
0,1,Carmencita,0,1.0,1894.0,short,0,0,0,0,...,0,0,1,0,0,0,0,0,1858,5.7
1,2,Le clown et ses chiens,0,5.0,1892.0,short,0,0,0,1,...,0,0,1,0,0,0,0,0,243,6.0
2,3,Pauvre Pierrot,0,4.0,1892.0,short,0,0,0,1,...,1,0,0,0,0,0,0,0,1629,6.5
3,4,Un bon bock,0,12.0,1892.0,short,0,0,0,1,...,0,0,1,0,0,0,0,0,158,6.0
4,5,Blacksmith Scene,0,1.0,1893.0,short,0,0,0,0,...,0,0,1,0,0,0,0,0,2455,6.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214286,9916690,Horrid Henry Delivers the Milk,0,10.0,2012.0,tvEpisode,0,0,1,1,...,0,0,0,0,0,0,0,0,5,6.6
1214287,9916720,The Nun 2,0,10.0,2019.0,short,0,0,0,0,...,0,0,0,0,0,0,0,0,184,5.5
1214288,9916730,6 Gunn,0,116.0,2017.0,movie,0,0,0,0,...,0,0,0,0,0,0,0,0,5,8.4
1214289,9916766,Episode #10.15,0,43.0,2019.0,tvEpisode,0,0,0,0,...,0,0,0,0,0,0,0,0,18,6.7


In [33]:
df.dropna(inplace=True)

In [34]:
X = df[df.columns[2:-1]].values
y = df['rating'].values

In [35]:
X

array([[0.000e+00, 1.000e+00, 1.894e+03, ..., 0.000e+00, 0.000e+00,
        1.858e+03],
       [0.000e+00, 5.000e+00, 1.892e+03, ..., 0.000e+00, 0.000e+00,
        2.430e+02],
       [0.000e+00, 4.000e+00, 1.892e+03, ..., 0.000e+00, 0.000e+00,
        1.629e+03],
       ...,
       [0.000e+00, 1.000e+01, 2.019e+03, ..., 0.000e+00, 0.000e+00,
        1.840e+02],
       [0.000e+00, 1.160e+02, 2.017e+03, ..., 0.000e+00, 0.000e+00,
        5.000e+00],
       [0.000e+00, 4.300e+01, 2.019e+03, ..., 0.000e+00, 0.000e+00,
        1.800e+01]])

In [36]:
y

array([5.7, 6. , 6.5, ..., 5.5, 8.4, 6.7])

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3)

In [39]:
X_tr.shape

(611900, 32)

In [40]:
X_te.shape

(262244, 32)

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
model = LinearRegression()
model.fit(X_tr, y_tr)

LinearRegression()

In [43]:
preds_tr = model.predict(X_tr)
preds_te = model.predict(X_te)

In [44]:
preds_te[:10]

array([6.92815394, 6.38935144, 6.40622975, 7.37826479, 6.93198575,
       6.8446341 , 7.41825309, 6.96227327, 6.67496977, 7.08568339])

In [45]:
y_te[:10].values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [46]:
y_te.mean()

6.870495797806623

In [47]:
from sklearn.metrics import mean_absolute_error

In [48]:
mean_absolute_error(y_tr, preds_tr)

0.9871155004673212

In [49]:
mean_absolute_error(y_te, preds_te)

0.9869357262637958